In [1]:
pip install opencv-python ultralytics deep_sort_realtime

   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/8.4 MB 8.5 MB/s eta 0:00:01
   ----------- ---------------------------- 2.4/8.4 MB 7.1 MB/s eta 0:00:01
   ----------------- ---------------------- 3.7/8.4 MB 6.8 MB/s eta 0:00:01
   ----------------------- ---------------- 5.0/8.4 MB 6.4 MB/s eta 0:00:01
   --------------------------- ------------ 5.8/8.4 MB 5.9 MB/s eta 0:00:01
   ----------------------------- ---------- 6.3/8.4 MB 5.4 MB/s eta 0:00:01
   ---------------------------------- ----- 7.3/8.4 MB 5.3 MB/s eta 0:00:01
   ---------------------------------------  8.4/8.4 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------- 8.4/8.4 MB 4.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
pip uninstall backports


Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import os
import time
from pathlib import Path
import random

# === CONFIGURATION ===
MODEL_PATH = Path(r'C:/Users/hp/Downloads/best.pt')
VIDEO_PATH = Path(r'C:/Users/hp/Downloads/15sec_input_720p.mp4')
OUTPUT_VIDEO_PATH = Path(rf'output_tracked_{int(time.time())}.mp4')

MAX_AGE = 60
N_INIT = 3
MAX_COSINE_DISTANCE = 0.2
YOLO_CONF_THRESH = 0.4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# === COLOR MEMORY ===
id_colors = {}
def get_color(track_id):
    if track_id not in id_colors:
        id_colors[track_id] = tuple(random.randint(0, 255) for _ in range(3))
    return id_colors[track_id]

# === LOAD MODEL ===
print("[INFO] Loading YOLOv11 model...")
try:
    model = YOLO(str(MODEL_PATH))
except Exception as e:
    raise RuntimeError(f"Error loading YOLO model: {e}")

# === TRACKER INIT ===
print(f"[INFO] Initializing DeepSort (max_age={MAX_AGE}, n_init={N_INIT})...")
tracker = DeepSort(max_age=MAX_AGE, n_init=N_INIT, max_cosine_distance=MAX_COSINE_DISTANCE)

# === VIDEO CAPTURE ===
print(f"[INFO] Opening video file: {VIDEO_PATH}")
cap = cv2.VideoCapture(str(VIDEO_PATH))
if not cap.isOpened():
    raise RuntimeError(f"Failed to open video: {VIDEO_PATH}")

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"[INFO] Video - Width: {width}, Height: {height}, FPS: {fps}, Frames: {frame_count}")

# === OUTPUT VIDEO ===
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(str(OUTPUT_VIDEO_PATH), fourcc, fps, (width, height))

# === TRACKING LOOP ===
frame_idx = 0
player_log = {}
start_time = time.time()

print("[INFO] Tracking started. Press 'q' to stop early.")
while True:
    ret, frame = cap.read()
    if not ret:
        print("[INFO] End of video reached.")
        break
    frame_idx += 1

    # Optional resizing to speed up (comment out if not needed)
    # frame = cv2.resize(frame, (width // 2, height // 2))

    try:
        results = model.predict(source=frame, conf=YOLO_CONF_THRESH, device=DEVICE)
    except Exception as e:
        print(f"[WARN] YOLO inference failed at frame {frame_idx}: {e}")
        continue

    detections = results[0].boxes.data.cpu().numpy()
    dets_for_tracker = []

    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        class_id = int(cls)
        class_name = model.names[class_id].lower()
        if class_name == 'player':
            bbox = [x1, y1, x2 - x1, y2 - y1]
            dets_for_tracker.append((bbox, conf, class_name))

    tracks = tracker.update_tracks(dets_for_tracker, frame=frame)

    for track in tracks:
        if not track.is_confirmed() or track.is_deleted():
            continue
        if track.time_since_update > 0:
            continue

        track_id = track.track_id
        x1, y1, x2, y2 = map(int, track.to_ltrb())
        color = get_color(track_id)

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        label = f"Player {track_id}"
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        if track_id not in player_log:
            player_log[track_id] = []
        player_log[track_id].append(frame_idx)

    out.write(frame)

    cv2.imshow("Player Re-ID & Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("[INFO] Interrupted by user.")
        break

    if frame_idx % 50 == 0:
        elapsed = time.time() - start_time
        print(f"[INFO] Processed {frame_idx}/{frame_count} frames ({frame_idx / elapsed:.2f} FPS)")

# === CLEANUP ===
cap.release()
out.release()
cv2.destroyAllWindows()

output_path = os.path.abspath(OUTPUT_VIDEO_PATH)
print(f"[INFO] Output video saved to: {output_path}")

log_path = Path('player_log.txt')
with open(log_path, 'w') as f:
    for pid, frames in player_log.items():
        f.write(f"Player {pid}: Frames {','.join(map(str, frames))}\n")

print(f"[INFO] Player log saved to: {log_path.resolve()}")
print("[INFO] All processing complete.")


[INFO] Loading YOLOv11 model...
[INFO] Initializing DeepSort (max_age=60, n_init=3)...
[INFO] Opening video file: C:\Users\hp\Downloads\15sec_input_720p.mp4
[INFO] Video - Width: 1280, Height: 720, FPS: 25.0, Frames: 375
[INFO] Tracking started. Press 'q' to stop early.

0: 384x640 1 ball, 16 players, 2 referees, 2104.0ms
Speed: 57.6ms preprocess, 2104.0ms inference, 31.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 players, 2 referees, 3062.2ms
Speed: 137.4ms preprocess, 3062.2ms inference, 40.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 players, 2 referees, 2254.1ms
Speed: 11.2ms preprocess, 2254.1ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 1 referee, 1960.3ms
Speed: 4.2ms preprocess, 1960.3ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 1748.0ms
Speed: 4.1ms preprocess, 1748.0ms inference, 2.3ms postprocess per image at sha